In [29]:
import pandas as pd
from pulp import *

In [30]:
distmatrix = pd.read_excel('warehouse_city.xlsx')
distmatrix

,Warehouse,city 1,city 2,city 3,city 4,city 5,city 6,city 7
0,city 1,0,2304,2528,950,906,947,1475
1,city 2,2304,0,2829,1776,2310,1475,2192
2,city 3,2528,2829,0,3123,1965,2134,1187
3,city 4,950,1776,3123,0,1699,1149,1926
4,city 5,906,2310,1965,1699,0,842,915
5,city 6,947,1475,2134,1149,842,0,922
6,city 7,1475,2192,1187,1926,915,922,0


In [31]:
distmatrix.set_index('Warehouse', inplace=True)
distmatrix

,city 1,city 2,city 3,city 4,city 5,city 6,city 7
Warehouse,,,,,,,
city 1,0,2304,2528,950,906,947,1475
city 2,2304,0,2829,1776,2310,1475,2192
city 3,2528,2829,0,3123,1965,2134,1187
city 4,950,1776,3123,0,1699,1149,1926
city 5,906,2310,1965,1699,0,842,915
city 6,947,1475,2134,1149,842,0,922
city 7,1475,2192,1187,1926,915,922,0


In [32]:
keys = [(w, c) for w in distmatrix.index for c in distmatrix.columns]  # list comprehension
keys

[('city 1', 'city 1'),
 ('city 1', 'city 2'),
 ('city 1', 'city 3'),
 ('city 1', 'city 4'),
 ('city 1', 'city 5'),
 ('city 1', 'city 6'),
 ('city 1', 'city 7'),
 ('city 2', 'city 1'),
 ('city 2', 'city 2'),
 ('city 2', 'city 3'),
 ('city 2', 'city 4'),
 ('city 2', 'city 5'),
 ('city 2', 'city 6'),
 ('city 2', 'city 7'),
 ('city 3', 'city 1'),
 ('city 3', 'city 2'),
 ('city 3', 'city 3'),
 ('city 3', 'city 4'),
 ('city 3', 'city 5'),
 ('city 3', 'city 6'),
 ('city 3', 'city 7'),
 ('city 4', 'city 1'),
 ('city 4', 'city 2'),
 ('city 4', 'city 3'),
 ('city 4', 'city 4'),
 ('city 4', 'city 5'),
 ('city 4', 'city 6'),
 ('city 4', 'city 7'),
 ('city 5', 'city 1'),
 ('city 5', 'city 2'),
 ('city 5', 'city 3'),
 ('city 5', 'city 4'),
 ('city 5', 'city 5'),
 ('city 5', 'city 6'),
 ('city 5', 'city 7'),
 ('city 6', 'city 1'),
 ('city 6', 'city 2'),
 ('city 6', 'city 3'),
 ('city 6', 'city 4'),
 ('city 6', 'city 5'),
 ('city 6', 'city 6'),
 ('city 6', 'city 7'),
 ('city 7', 'city 1'),
 ('city 7',

In [33]:
dist_dict = {(w, c): distmatrix.loc[w, c] for w in distmatrix.index for c in
             distmatrix.columns}  # dictionary comprehension
dist_dict

{('city 1', 'city 1'): 0,
 ('city 1', 'city 2'): 2304,
 ('city 1', 'city 3'): 2528,
 ('city 1', 'city 4'): 950,
 ('city 1', 'city 5'): 906,
 ('city 1', 'city 6'): 947,
 ('city 1', 'city 7'): 1475,
 ('city 2', 'city 1'): 2304,
 ('city 2', 'city 2'): 0,
 ('city 2', 'city 3'): 2829,
 ('city 2', 'city 4'): 1776,
 ('city 2', 'city 5'): 2310,
 ('city 2', 'city 6'): 1475,
 ('city 2', 'city 7'): 2192,
 ('city 3', 'city 1'): 2528,
 ('city 3', 'city 2'): 2829,
 ('city 3', 'city 3'): 0,
 ('city 3', 'city 4'): 3123,
 ('city 3', 'city 5'): 1965,
 ('city 3', 'city 6'): 2134,
 ('city 3', 'city 7'): 1187,
 ('city 4', 'city 1'): 950,
 ('city 4', 'city 2'): 1776,
 ('city 4', 'city 3'): 3123,
 ('city 4', 'city 4'): 0,
 ('city 4', 'city 5'): 1699,
 ('city 4', 'city 6'): 1149,
 ('city 4', 'city 7'): 1926,
 ('city 5', 'city 1'): 906,
 ('city 5', 'city 2'): 2310,
 ('city 5', 'city 3'): 1965,
 ('city 5', 'city 4'): 1699,
 ('city 5', 'city 5'): 0,
 ('city 5', 'city 6'): 842,
 ('city 5', 'city 7'): 915,
 ('city

In [34]:
demand_dict = dict(zip(distmatrix.columns, [10000, 20000, 33000, 9000, 60000, 2500, 35000]))
demand_dict

{'city 1': 10000,
 'city 2': 20000,
 'city 3': 33000,
 'city 4': 9000,
 'city 5': 60000,
 'city 6': 2500,
 'city 7': 35000}

In [35]:
warehouse = distmatrix.index
customers = distmatrix.columns

In [36]:
flows = LpVariable.dicts('flows', keys, cat='Binary')
open_w = LpVariable.dicts('open_w', distmatrix.index, cat='Binary')

In [37]:
# creating a model for running optimization
model = LpProblem('transportprob', sense=LpMinimize)

# defining the objective function
model += lpSum([demand_dict.get(c) * flows[(w, c)] * dist_dict.get((w, c)) for w in open_w for c in distmatrix.columns])

In [38]:
# defining constraints
for c in customers:
    model += lpSum([flows.get((w, c)) for w in warehouse]) == 1

model += lpSum([open_w[w] for w in warehouse]) == 3

for w in warehouse:
    for c in customers:
        model += open_w[w] >= flows.get((w,c))

In [39]:
model.solve()

1

In [40]:
for i in open_w:
    print (open_w[i])

open_w_city_1
open_w_city_2
open_w_city_3
open_w_city_4
open_w_city_5
open_w_city_6
open_w_city_7


In [41]:
for w,c in flows:
    print (flows[(w,c)])

flows_('city_1',_'city_1')
flows_('city_1',_'city_2')
flows_('city_1',_'city_3')
flows_('city_1',_'city_4')
flows_('city_1',_'city_5')
flows_('city_1',_'city_6')
flows_('city_1',_'city_7')
flows_('city_2',_'city_1')
flows_('city_2',_'city_2')
flows_('city_2',_'city_3')
flows_('city_2',_'city_4')
flows_('city_2',_'city_5')
flows_('city_2',_'city_6')
flows_('city_2',_'city_7')
flows_('city_3',_'city_1')
flows_('city_3',_'city_2')
flows_('city_3',_'city_3')
flows_('city_3',_'city_4')
flows_('city_3',_'city_5')
flows_('city_3',_'city_6')
flows_('city_3',_'city_7')
flows_('city_4',_'city_1')
flows_('city_4',_'city_2')
flows_('city_4',_'city_3')
flows_('city_4',_'city_4')
flows_('city_4',_'city_5')
flows_('city_4',_'city_6')
flows_('city_4',_'city_7')
flows_('city_5',_'city_1')
flows_('city_5',_'city_2')
flows_('city_5',_'city_3')
flows_('city_5',_'city_4')
flows_('city_5',_'city_5')
flows_('city_5',_'city_6')
flows_('city_5',_'city_7')
flows_('city_6',_'city_1')
flows_('city_6',_'city_2')
f

In [51]:
# generating the list of keys building the variable dict
warehouselist =[]
for i in open_w:
    warehouselist.append(open_w[i])
# generating the list of associated values building the variable dict
warehouseopenvalues = []
for i in open_w:
    warehouseopenvalues.append(open_w[i].varValue)

#creating a dictionary for output
new_data = pd.DataFrame({"warehouse name":warehouselist,"warehouse open":warehouseopenvalues})
new_data

,warehouse name,warehouse open
0,open_w_city_1,0
1,open_w_city_2,1
2,open_w_city_3,1
3,open_w_city_4,0
4,open_w_city_5,1
5,open_w_city_6,0
6,open_w_city_7,0


In [43]:
value(model.objective)

58481000